In [1]:
import os

import rmgpy.species
import rmgpy.chemkin

import autotst.species
import autotst.reaction
import autotst.calculator.gaussian

import pandas as pd

import logging
from hotbit import Hotbit

import ase.calculators.lj

import ase.io
import glob
import job_manager

utils.py:147 _init_num_threads INFO Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
utils.py:159 _init_num_threads INFO NumExpr defaulting to 8 threads.


In [2]:
def get_label(rmg_rxn):
    label = ''
    for reactant in rmg_rxn.reactants:
        label += f'{reactant.smiles}+'
    label = label[:-1]
    label += '_'
    for product in rmg_rxn.products:
        label += f'{product.smiles}+'
    label = label[:-1]
    return label

In [3]:
def reaction2SMILES(reaction):
    string = ""
    for react in reaction.reactants:
        if isinstance(react, rmgpy.species.Species):
            string += f"{react.molecule[0].to_smiles()}+"
        elif isinstance(react, rmgpy.molecule.Molecule):
            string += f"{react.to_smiles()}+"
    string = string[:-1]
    string += "_"
    for prod in reaction.products:
        if isinstance(prod, rmgpy.species.Species):
            string += f"{prod.molecule[0].to_smiles()}+"
        elif isinstance(prod, rmgpy.molecule.Molecule):
            string += f"{prod.to_smiles()}+"
    label = string[:-1]
    return label

In [4]:
try:
    DFT_DIR = os.environ['DFT_DIR']
except KeyError:
    DFT_DIR = '/work/westgroup/harris.se/autoscience/autoscience_workflow/results/dft'


reaction_index = 1
# reaction_index = 168
# reaction_index = int(sys.argv[1])
# print(f'Reaction index is {reaction_index}')


# Load the species from the official species list
# scripts_dir = os.path.dirname(__file__)
scripts_dir = '/work/westgroup/harris.se/autoscience/autoscience_workflow/workflow/scripts'
reaction_csv = os.path.join(scripts_dir, '..', '..', 'resources', 'reaction_list.csv')
reaction_df = pd.read_csv(reaction_csv)


reaction_smiles = reaction_df.SMILES[reaction_index]
print(reaction_smiles)

[H][H]+[O]_[H]+[OH]


In [5]:
reaction = autotst.reaction.Reaction(label=reaction_smiles)

In [6]:
print(reaction)

<Reaction "[H][H]+[O]_[H]+[OH]">


In [7]:
reaction.ts['forward'][0].get_molecules()

reaction.py:167 load_databases INFO Loading RMG database from '/home/harris.se/rmg/RMG-database/input'
transport.py:294 load_groups INFO Loading transport group database from /home/harris.se/rmg/RMG-database/input/transport/groups...
statmech.py:541 load_libraries INFO Loading frequencies library from halogens_G4.py in /home/harris.se/rmg/RMG-database/input/statmech/libraries...
statmech.py:555 load_groups INFO Loading frequencies group database from /home/harris.se/rmg/RMG-database/input/statmech/groups...
thermo.py:939 load_libraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /home/harris.se/rmg/RMG-database/input/thermo/libraries...
thermo.py:939 load_libraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /home/harris.se/rmg/RMG-database/input/thermo/libraries...
thermo.py:939 load_libraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /home/harris.se/rmg/RMG-database/input/thermo/libraries...
thermo.py:970 load_group

(<rdkit.Chem.rdchem.Mol at 0x2b206f8d2350>, Atoms(symbols='OH2', pbc=False))

In [8]:
print(reaction.ts['forward'][0].get_xyz_block())

O       1.068600000000000     -0.089100000000000      0.000000000000000
H      -0.933500000000000     -0.128600000000000      0.000000000000000
H      -0.135100000000000      0.217700000000000      0.000000000000000



In [9]:
# reaction.ts_databases['H_Abstraction'].estimate_distances(reaction.rmg_reaction).distances

In [10]:
reaction.get_labeled_reaction()

reaction.py:340 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "H_Abstraction">
reaction.py:368 get_labeled_reaction INFO Matched reaction to H_Abstraction family
reaction.py:340 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "R_Addition_MultipleBond">
reaction.py:340 get_labeled_reaction INFO Trying to match reacction to <ReactionFamily "intra_H_migration">
reaction.py:351 get_labeled_reaction ERROR Couldn't match <Molecule "[H][H]"> + <Molecule "[O]"> <=> <Molecule "[H]"> + <Molecule "[OH]"> to intra_H_migration, trying different combination...


(TemplateReaction(reactants=[Molecule(smiles="[H][H]"), Molecule(smiles="[O]")], products=[Molecule(smiles="[OH]"), Molecule(smiles="[H]")], pairs=[[Molecule(smiles="[O]"), Molecule(smiles="[OH]")], [Molecule(smiles="[H][H]"), Molecule(smiles="[H]")]], family='H_Abstraction', template=['H2', 'O_atom_triplet']),
 'H_Abstraction')

In [11]:
print(reaction.ts['forward'][0].get_xyz_block())

O       1.068600000000000     -0.089100000000000      0.000000000000000
H      -0.933500000000000     -0.128600000000000      0.000000000000000
H      -0.135100000000000      0.217700000000000      0.000000000000000



In [ ]:
print(reaction.ts['reverse'][0].get_xyz_block())

In [ ]:
print(reaction.rmg_reaction)
print(reaction.rmg_reaction.family)

In [ ]:
reaction.get_rmg_complexes()

In [ ]:
# TODO handle multiple forward TS?
# TODO also calculate reverse TS?
# reaction.generate_conformers(ase_calculator=Hotbit())
reaction.generate_conformers(ase_calculator=ase.calculators.lj.LennardJones())

In [ ]:
# These distances are way off. Why?
print(reaction.ts['forward'][0].get_xyz_block())

In [ ]:
print(reaction.ts['reverse'][0].get_xyz_block())

In [21]:
reaction_base_dir = os.path.join(DFT_DIR, 'kinetics', f'reaction_{reaction_index:04}')
os.makedirs(reaction_base_dir, exist_ok=True)

In [ ]:
shell_dir = os.path.join(reaction_base_dir, 'shell')
os.makedirs(shell_dir, exist_ok=True)
# Do the shell calculation
# write Gaussian input files
for i, ts in enumerate(reaction.ts['forward']):
    gaussian = autotst.calculator.gaussian.Gaussian(conformer=ts)
    calc = gaussian.get_shell_calc()
    calc.label = f'fwd_ts_{i:04}'

    calc.directory = shell_dir
    calc.parameters.pop('scratch')
    calc.parameters.pop('multiplicity')
    calc.parameters['mult'] = ts.rmg_molecule.multiplicity

    calc.write_input(ts.ase_molecule)

print("done")

In [ ]:
# make the shell slurm script
slurm_run_file = os.path.join(shell_dir, 'run.sh')
slurm_settings = {
    '--job-name': f'g16_shell_{reaction_index}',
    '--error': 'error.log',
    '--output': 'output.log',
    '--nodes': 1,
    '--partition': 'west,short',
    '--exclude': 'c5003',
    '--mem': '20Gb',
    '--time': '24:00:00',
    '--cpus-per-task': 16,
}

slurm_file_writer = job_manager.SlurmJobFile(
    full_path=slurm_run_file,
)
slurm_file_writer.settings = slurm_settings
slurm_file_writer.content = [
    'export GAUSS_SCRDIR=/scratch/harris.se/guassian_scratch\n',
    'mkdir -p $GAUSS_SCRDIR\n',
    'module load gaussian/g16\n',
    'source /shared/centos7/gaussian/g16/bsd/g16.profile\n\n',

    f'g16 fwd_ts_{i:04}.com\n',
]
slurm_file_writer.write_file()


In [12]:
import ase.io.gaussian

In [13]:
# once the shell calculation is done, load the geometry and run the overall TS optimization
shell_dir = '/work/westgroup/harris.se/autoscience/autoscience_workflow/results/dft/kinetics/reaction_0001/shell'
shell_opt = os.path.join(shell_dir, 'fwd_ts_0000.log')
print(reaction.ts['forward'][0].get_xyz_block())
with open(shell_opt, 'r') as f:
    atoms = ase.io.gaussian.read_gaussian_out(f)
print(atoms.get_positions())

O       1.068600000000000     -0.089100000000000      0.000000000000000
H      -0.933500000000000     -0.128600000000000      0.000000000000000
H      -0.135100000000000      0.217700000000000      0.000000000000000

[[ 0.        0.581367  0.      ]
 [ 0.145204 -2.66399   0.      ]
 [-0.145204 -1.986949 -0.      ]]


In [16]:
reaction.ts['forward'][0]._ase_molecule

Atoms(symbols='OH2', pbc=False)

In [14]:
from ase.visualize import view
import matplotlib
matplotlib.use('agg')

In [ ]:
view(reaction.ts['forward'][0]._ase_molecule, viewer='ngl')

In [ ]:
view(atoms, viewer='ngl')

In [18]:
# once the shell calculation is done, load the geometry and run the overall TS optimization
shell_dir = '/work/westgroup/harris.se/autoscience/autoscience_workflow/results/dft/kinetics/reaction_0001/shell'
shell_opt = os.path.join(shell_dir, 'shell_debug', 'fwd_ts_0000.log')
with open(shell_opt, 'r') as f:
    atoms2 = ase.io.gaussian.read_gaussian_out(f)
print(atoms.get_positions())

[[ 0.        0.581367  0.      ]
 [ 0.145204 -2.66399   0.      ]
 [-0.145204 -1.986949 -0.      ]]


In [ ]:
view(atoms2, viewer='ngl')

In [ ]:
# In theory, if all degrees of freedom haven't been frozen, then we read in the geometry
# but for this simplistic case, proceed to the overall calculation

In [25]:
overall_dir = os.path.join(reaction_base_dir, 'overall')
os.makedirs(overall_dir, exist_ok=True)
# Do the shell calculation
# write Gaussian input files
for i, ts in enumerate(reaction.ts['forward']):
    gaussian = autotst.calculator.gaussian.Gaussian(conformer=ts)
    calc = gaussian.get_overall_calc()
    calc.label = f'fwd_ts_{i:04}'

    calc.directory = overall_dir
    calc.parameters.pop('scratch')
    calc.parameters.pop('multiplicity')
    calc.parameters['mult'] = ts.rmg_molecule.multiplicity

    calc.write_input(ts.ase_molecule)

print("done")

done


In [26]:
# make the overall slurm script
slurm_run_file = os.path.join(overall_dir, 'run.sh')
slurm_settings = {
    '--job-name': f'ts_{reaction_index}',
    '--error': 'error.log',
    '--output': 'output.log',
    '--nodes': 1,
    '--partition': 'short',
    '--mem': '20Gb',
    '--time': '24:00:00',
    '--cpus-per-task': 48,
}

slurm_file_writer = job_manager.SlurmJobFile(
    full_path=slurm_run_file,
)
slurm_file_writer.settings = slurm_settings
slurm_file_writer.content = [
    'export GAUSS_SCRDIR=/scratch/harris.se/guassian_scratch\n',
    'mkdir -p $GAUSS_SCRDIR\n',
    'module load gaussian/g16\n',
    'source /shared/centos7/gaussian/g16/bsd/g16.profile\n\n',

    f'g16 fwd_ts_{i:04}.com\n',
]
slurm_file_writer.write_file()


In [ ]:

spec = autotst.species.Species([species_smiles])

print(f"loaded species {species_smiles}")
thermo_base_dir = os.path.join(DFT_DIR, 'thermo')
species_base_dir = os.path.join(thermo_base_dir, f'species_{species_index:04}')
os.makedirs(species_base_dir, exist_ok=True)


# generate conformers
spec.generate_conformers(ase_calculator=Hotbit())
n_conformers = len(spec.conformers[species_smiles])
print(f'{n_conformers} found with Hotbit')


In [ ]:
# load the model
chemkin_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/chem_annotated.inp"
dictionary_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/species_dictionary.txt"
transport_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/tran.dat"
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(
    chemkin_path,
    dictionary_path=dictionary_path,
    transport_path=transport_path
)
print(f"Loaded model with {len(species_list)} species and {len(reaction_list)} reactions")


rxn_idx = 186  # R recombo
rxn_idx = 194
rxn_idx = 274 # first H abstraction

# rxn_idx = 236 another H abstraction



In [ ]:
rmg_rxn = reaction_list[rxn_idx]
print(rmg_rxn)
print(rmg_rxn.family)

In [ ]:
kinetics_base_dir = '/work/westgroup/harris.se/autoscience/autoscience_workflow/results/dft/kinetics'
reaction_base_dir = os.path.join(kinetics_base_dir, f'reaction_{rxn_idx:04}')
os.makedirs(kinetics_base_dir, exist_ok=True)

In [ ]:
# TODO fix reaction index


In [ ]:
label = get_label(rmg_rxn)
print("label", label)

reaction = autotst.reaction.Reaction(label=label, rmg_reaction=rmg_rxn)
print("reaction", reaction)

reaction.get_labeled_reaction()

print("Got labeled reaction")
transition_states = reaction.ts["reverse"]


print("ts0", transition_states[0])

print("rxn.ts", reaction.ts)
print("About to start hotbit")
logging.warning("Danger zone 0")
# reaction.generate_conformers_all(ase_calculator=Hotbit())
reaction.generate_conformers(ase_calculator=Hotbit())

print("ran hotbit")

for ts in reaction.ts['forward']:
    print(ts)


# overall calc
ts_dir = os.path.join(reaction_base_dir, 'ts')

In [ ]:
# write Gaussian input files
for i, ts in enumerate(reaction.ts['forward']):
    gaussian = autotst.calculator.gaussian.Gaussian(conformer=ts)
    calc = gaussian.get_overall_calc()
    calc.label = f'fwd_ts_{i:04}'

    calc.directory = ts_dir
    calc.parameters.pop('scratch')
    calc.parameters.pop('multiplicity')
    calc.parameters['mult'] = ts.rmg_molecule.multiplicity

    calc.write_input(ts.ase_molecule)

print("done")

In [ ]:
n_ts = len(reaction.ts['forward'])
print(f'{n_ts} found with Hotbit')

# Make a file to run Gaussian
slurm_run_file = os.path.join(ts_dir, 'run.sh')
slurm_settings = {
    '--job-name': f'g16_ts_{rxn_idx}',
    '--error': 'error.log',
    '--output': 'output.log',
    '--nodes': 1,
    '--partition': 'west,short',
    '--mem': '20Gb',
    '--time': '24:00:00',
    '--cpus-per-task': 16,
    '--array': f'0-{n_conformers}%40',
}

slurm_file_writer = job_manager.SlurmJobFile(
    full_path=slurm_run_file,
)
slurm_file_writer.settings = slurm_settings

In [ ]:
slurm_file_writer.content = [
    'export GAUSS_SCRDIR=/scratch/harris.se/guassian_scratch\n',
    'mkdir -p $GAUSS_SCRDIR\n',
    'module load gaussian/g16\n',
    'source /shared/centos7/gaussian/g16/bsd/g16.profile\n\n',

    'RUN_i=$(printf "%04.0f" $(($SLURM_ARRAY_TASK_ID)))\n',
    'fname="fwd_ts_${RUN_i}.com"\n\n',

    'g16 $fname\n',
]
slurm_file_writer.write_file()

# submit the job
gaussian_conformers_job = job_manager.SlurmJob()
slurm_cmd = f"sbatch {slurm_run_file}"
gaussian_conformers_job.submit(my_cmd)
